In [2]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from flask import Flask, request, render_template
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import load_model


# Initialize Flask app
app = Flask(__name__)

# Preprocessing the dataset
def preprocess_data(data):
    features = [
        'category', 'tags', 'location', 'cuisine_type', 'attraction_type',
        'family_friendly', 'star_rating', 'best_time_to_visit', 'wheelchair_accessible', 'amenities'
    ]

    # Combine features into a single text column
    data['combined_features'] = data[features].fillna('').apply(
        lambda x: ' '.join(x.map(str).str.lower().map(lambda s: re.sub(r'\W+', ' ', s))),
        axis=1
    )
    return data


# Load TF-IDF vectorizer
def load_vectorizer():
    with open('vectorizer.pkl', 'rb') as f:
        return pickle.load(f)


# Recommendation function
def recommend_with_ann(input_text, data, model, vectorizer, top_n=10):
    input_vec = vectorizer.transform([input_text]).todense()
    numerical_data = data[['rating', 'reviews_count', 'popularity_score']].fillna(0).to_numpy()
    input_vec_repeated = np.repeat(input_vec, numerical_data.shape[0], axis=0)
    combined_input = np.hstack([input_vec_repeated, numerical_data])
    predictions = model.predict(combined_input)
    data['score'] = predictions.flatten()
    recommendations = data.sort_values(by='score', ascending=False).head(top_n)
    return recommendations


# Flask route to handle form submission
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        # Load dataset and preprocess
        data = pd.read_csv('mauritiusDataset.csv')  # Ensure this path is correct
        data = preprocess_data(data)

        # Load vectorizer and model
        vectorizer = load_vectorizer()
        model = load_model('ann_model_improved.keras')

        # Get form inputs
        tags = request.form.get('tags', '').strip().lower()
        category = request.form.get('category', '').strip().lower()
        cuisine_type = request.form.get('cuisine_type', '').strip().lower()
        amenities = request.form.get('amenities', '').strip().lower()
        wheelchair = request.form.get('wheelchair', 'no').strip().lower()
        rating = float(request.form.get('rating', 0))

        # Combine input text for TF-IDF
        input_text = f"{tags} {category} {cuisine_type} {amenities}"

        # Filter data for the input rating
        filtered_data = data[data['rating'] >= rating]

        # Filter for wheelchair accessibility if required
        if wheelchair == 'yes':
            filtered_data = filtered_data[filtered_data['wheelchair_accessible'].str.contains('yes', case=False, na=False)]

        # Check if there is data to recommend
        if filtered_data.empty:
            return render_template('index.html', recommendations=None, error="No recommendations found.")

        # Generate recommendations
        recommendations = recommend_with_ann(input_text, filtered_data, model, vectorizer)

        # Columns to display in the table
        display_columns = ['name', 'category', 'rating', 'address', 'imageUrls', 'latitude', 'longitude', 'url', 'popularity_score']
        recommendations = recommendations[display_columns]

        # Pass recommendations to the template
        return render_template('index.html', recommendations=recommendations.to_dict(orient='records'))

    return render_template('index.html', recommendations=None)


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1